Carpages scraping project
carpages.ca is a popular canadian website for buying used and new vehicles in canada. In this project, I'll be scraping the data using 
python libraries such as BeautifulSoup, requests and pandas   

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib

In [36]:
# Lets check to see if we can get a response from the website
#website = "https://www.kijijiautos.ca/cars/"
website = "https://www.carpages.ca/used-cars/search/?num_results=50&category_id=5&p=3"
response = requests.get(website)

In [37]:
response.status_code

200

In [38]:
soup = BeautifulSoup(response.content,'html.parser')

In [39]:
soup


<!DOCTYPE html>

<!--
                                ______________
                        __..=='|'   |         ``-._
           \=====_..--'/'''    |    |              ``-._
           |'''''      ```---..|____|_______________[)>.``-.._____
           |\_______.....__________|____________     ''  \      __````---.._
         ./'     /.-'_'_`-.\       |  ' '       ```````---|---/.-'_'_`=.-.__```-._
         |.__  .'/ /     \ \`.      \                     | .'/ /     \ \`. ```-- `.
          \  ``|| |   o   | ||-------\-------------------/--|| |   o   | ||--------|
           "`--' \ \ _ _ / / |______________________________| \ \ _ _ / / |..----```
                  `-.....-'                                    `-.....-'

               ______
              / ____/___ __________  ____ _____ ____  _____ _________ _
             / /   / __ `/ ___/ __ \/ __ `/ __ `/ _ \/ ___// ___/ __ `/
            / /___/ /_/ / /  / /_/ / /_/ / /_/ /  __(__  )/ /__/ /_/ /
            \____/\__,_/_/  

In [43]:
cars = soup.find_all('div', class_='media soft push-none rule')
dealer = cars[0].find_all("h5")
dealer[-1]

<h5 class="hN">Auto Flash BFH</h5>

In [7]:
mileage_string = car.find("div",class_="grey l-column l-column--small-6 l-column--medium-4").find_all("span")
mileage_num = ""
for m in mileage_string:
    if m != "K" or m != "M":
        mileage_num += m.text

print(mileage_num)
mileage = int(mileage_num[0:-2].replace(",",""))
mileage

NameError: name 'car' is not defined

In [114]:
location = "North York, ON"
city,province = location[0:-4],location[-2:len(location)]
city

'North York'

In [63]:
def getMileage(car):
    # Takes a car and gives us the mileage from the tag
    unFilteredMileageTags = car.find("div",class_="grey l-column l-column--small-6 l-column--medium-4").find_all("span")
    mileage = ""
    for tag in unFilteredMileageTags:
        mileage += tag.text
   
    try:
        mileage = int(mileage[0:-2].replace(",",""))
    except ValueError:
        mileage = None
    return mileage

def getLocationField(car):
    location = car.find("p").text
    city,province = location[0:-4],location[-2:len(location)]
    return city,province

def getYearMake(car):
    # gets the year and make of the vehicle
    vehicle = car.find_all("a")[1].text
    year = int(vehicle[0:5])
    make = vehicle[5:len(vehicle)]
    return year,make

def parse_page(no_listings,body_type,page_num):
    # page_num is the page number
    # result_num is the number of the 
    styles = {
        "1":"Convertable",
        "2":"Coupe",
        "3":"Hatchback",
        "4":"Hybrid/Electric",
        "5":"Sedan",
        "6":"SUV",
        "7":"Crossover",
        "8":"Van/Minivan",
        "9":"Pickup Truck"}

    website = "https://www.carpages.ca/used-cars/search/?num_results="+str(no_listings)+"&category_id="+str(body_type)+"&p="+str(page_num)
    response = requests.get(website)
    status = response.status_code
    soup = BeautifulSoup(response.content,'html.parser')

    titles,prices,links,mileages,dealers,cities,provinces,years,makes = [],[],[],[],[],[],[],[],[]
    conditions = []
    if status == 200:
        # this means we have the ability to scrape the website
        cars = soup.find_all('div', class_='media soft push-none rule')
        for car in cars:
            # We need to get the following data, title,price,link,mileage,dealer,location(city), location(province)
            title = car.find("a").get("title") # from the title (we can see if we can filter the model and the make from the )
            condition = title[0:4]
            year,make = getYearMake(car)
            try:
                price = float(car.find("strong").text.strip().replace("$","").replace(",",""))
            except ValueError:
                price = None
            link = "https://www.carpages.ca"+car.find("a").get("href")
            mileage = getMileage(car) # We make a function for the mileage as it takes some work
            dealer = car.find_all("h5")[-1].text
            city,province = getLocationField(car) # We make a similar function to parse the city and the province
            location = car.find("p").text
            
            conditions.append(condition)
            years.append(year)
            makes.append(make)
            titles.append(title)
            prices.append(price)
            links.append(link)
            mileages.append(mileage)
            dealers.append(dealer)
            cities.append(city)
            provinces.append(province)

    data_dict = {"Titles":titles,"Year":years,"Model":makes,"Style":styles[body_type],"Condition":conditions,"Price":prices,"Link":links,"Mileages":mileages,"Dealer":dealers,"City":cities,"Province":provinces}
    data = pd.DataFrame(data_dict)
    return data

# sedans are 5, 50 cars, 1 result
data = parse_page("50","5","3")


In [64]:
data

,Titles,Year,Model,Style,Condition,Price,Link,Mileages,Dealer,City,Province
0,Used 2020 Toyota Camry SE Auto for sale in Nor...,2020,Toyota Camry,Sedan,Used,29692.0,https://www.carpages.ca/used-cars/ontario/nort...,NaN,North Bay Toyota,North Bay,ON
1,Used 2020 Toyota Corolla LE CVT for sale in No...,2020,Toyota Corolla,Sedan,Used,25468.0,https://www.carpages.ca/used-cars/ontario/nort...,NaN,North Bay Toyota,North Bay,ON
2,Used 2018 Honda Clarity Plug-In Hybrid TOURING...,2018,Honda Clarity,Sedan,Used,27450.0,https://www.carpages.ca/used-cars/quebec/saint...,84493.0,Auto Flash BFH,Saint-Hubert,QC
3,"Used 2017 Nissan Sentra SV, 1.8L, PUSHBUTTON-S...",2017,Nissan Sentra,Sedan,Used,10450.0,https://www.carpages.ca/used-cars/quebec/saint...,179799.0,Auto Flash BFH,Saint-Hubert,QC
4,"Used 2015 Hyundai Sonata GLS, CAMERA-RECUL, BL...",2015,Hyundai Sonata,Sedan,Used,13888.0,https://www.carpages.ca/used-cars/quebec/saint...,125065.0,Auto Flash BFH,Saint-Hubert,QC
5,Used 2020 Acura TLX 2.4L P-AWS Tech A-Spec for...,2020,Acura TLX,Sedan,Used,34995.0,https://www.carpages.ca/used-cars/ontario/mark...,47622.0,Markham Acura,Markham,ON
6,Used 2022 Toyota Corolla XSE for sale in Whitb...,2022,Toyota Corolla,Sedan,Used,31900.0,https://www.carpages.ca/used-cars/ontario/whit...,16704.0,Whitby Toyota,Whitby,ON
7,Used 2009 Nissan Altima 4DR SDN I4 CVT 2.5 S f...,2009,Nissan Altima,Sedan,Used,8995.0,https://www.carpages.ca/used-cars/ontario/kitc...,110000.0,Vendora Credit Inc,Kitchener,ON
8,Used 2009 Nissan Altima 4DR SDN I4 CVT 2.5 S f...,2009,Nissan Altima,Sedan,Used,8995.0,https://www.carpages.ca/used-cars/ontario/anca...,110000.0,Village Auto of Ancaster,Ancaster,ON
9,Used 2021 Kia K5 GT-Line GT-LINE | AWD | LEATH...,2021,Kia K5,Sedan,Used,33988.0,https://www.carpages.ca/used-cars/ontario/kitc...,19107.0,Kitchener Hyundai,Kitchener,ON


In [47]:
def createDataBase(car,pages):
    # Lets create a DataBase
    frames = []
    for i in range(1,pages):
        print("Page"+str(i))
        df = parse_page("50",car,str(i))
        frames.append(df)
    database = pd.concat(frames)

    return database
database = createDataBase("5")

Page1
Page2
Page3
Page4
Page5
Page6
Page7
Page8
Page9
Page10
Page11
Page12
Page13
Page14
Page15
Page16
Page17
Page18
Page19
Page20
Page21
Page22
Page23
Page24
Page25
Page26
Page27
Page28
Page29
Page30
Page31
Page32
Page33
Page34
Page35
Page36
Page37
Page38
Page39
Page40
Page41
Page42
Page43
Page44
Page45
Page46
Page47
Page48
Page49
Page50
Page51
Page52
Page53
Page54
Page55
Page56
Page57
Page58
Page59
Page60
Page61
Page62
Page63
Page64
Page65
Page66
Page67
Page68
Page69
Page70
Page71
Page72
Page73
Page74
Page75
Page76
Page77
Page78
Page79
Page80
Page81
Page82
Page83
Page84
Page85
Page86
Page87
Page88
Page89
Page90
Page91
Page92
Page93
Page94
Page95
Page96
Page97
Page98
Page99
Page100
Page101
Page102
Page103
Page104
Page105
Page106
Page107
Page108
Page109
Page110
Page111
Page112
Page113
Page114
Page115
Page116
Page117
Page118
Page119
Page120
Page121
Page122
Page123
Page124
Page125
Page126
Page127
Page128
Page129
Page130
Page131
Page132
Page133
Page134
Page135
Page136
Page137
Page138
Page

In [74]:
bob = (database["Province"].value_counts()) 
bob.plot.hist

<bound method PlotAccessor.hist of <pandas.plotting._core.PlotAccessor object at 0x000001EAEC5D59D0>>